In [38]:
import json
import timeit
import tiktoken

In [39]:
!python setup.py build_ext --inplace

running build_ext

error: could not delete 'trie.cp310-win_amd64.pyd': Access is denied



copying build\lib.win-amd64-3.10\trie.cp310-win_amd64.pyd -> 


In [40]:
import trie

def bytes_to_unicode():
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

import time

class LinearTokenizer():
    def __init__(self, vocab):
        byte_encoder = bytes_to_unicode()
        byte_decoder = {v:k for k, v in byte_encoder.items()}

        self.vocab_encode = vocab
        self.vocab_decode = {}

        self.trie = trie.Trie()
        for token, token_id in self.vocab_encode.items():
            token_bytes = bytes([byte_decoder[c] for c in token])
            self.vocab_decode[token_id] = token_bytes
            self.trie.insert(token_bytes, token_id)

    def encode(self, text, return_token_tuple=False):
        # TODO special handling of <|endoftext|> token
        t0 = time.perf_counter()
        text = text.encode('utf-8')
        # print("UTF8 time:", time.perf_counter() - t0)

        t0 = time.perf_counter()
        ids = self.trie.encode(text)
        # print("Decode time:", time.perf_counter() - t0)
        return (ids, [self.vocab_decode[id] for id in ids]) if return_token_tuple else ids

    def decode(self, ids):
        out = bytes()
        for id in ids:
            if not id in self.vocab_decode:
                raise Exception(f"Error decoding {id}")
            out += self.vocab_decode[id]
        return out.decode('utf-8', errors="replace")


with open('vocab.json', 'r', encoding='utf-8') as f:
    tokenizer_config = json.load(f)
tokenizer_linear = LinearTokenizer(tokenizer_config)
tokenizer_tt = tiktoken.get_encoding("gpt2")

In [42]:
with open('../embedding-explorer/tokenizers/taylorswift.txt', 'r') as file:
    content = file.read()

ids_lin = tokenizer_linear.encode(content)
ids_tt = tokenizer_tt.encode(content)

decoded = tokenizer_linear.decode(ids_lin)
token_count = len(ids_lin)

print(token_count)
print(content == decoded)

48955
True


In [43]:
print("Linear:", timeit.timeit(lambda: tokenizer_linear.encode(content), number=1000))
print("Tiktoken:", timeit.timeit(lambda: tokenizer_tt.encode(content), number=1000))

Linear: 10.888645000000906
Tiktoken: 30.02101109999967
